In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

FMP_API_KEY = 'YOUR_FINANCIAL_MODELING_PREP_API_KEY'
ticker = 'BHP'

# Función para obtener cualquier endpoint
def get_fmp_data(endpoint, ticker):
    url = f'https://financialmodelingprep.com/api/v3/{endpoint}/{ticker}?apikey={FMP_API_KEY}'
    response = requests.get(url)
    return response.json()


In [2]:
profile = get_fmp_data('profile', ticker)
ratios = get_fmp_data('ratios', ticker)
income_statement = get_fmp_data('income-statement', ticker)
balance_sheet = get_fmp_data('balance-sheet-statement', ticker)
key_metrics = get_fmp_data('key-metrics', ticker)

In [3]:
def calcular_roic_wacc_fmp(income_list, balance_list, ratios_list, metrics_list):
    income = income_list[0]
    balance = balance_list[0]
    ratios = ratios_list[0]
    metrics = metrics_list[0]

    # Usamos operatingIncome como proxy de EBIT si no hay 'ebit'
    EBIT = income.get('ebit') or income.get('operatingIncome')
    if EBIT is None:
        raise ValueError("No se encontró EBIT ni operatingIncome en income statement.")

    tax_rate = ratios.get('effectiveTaxRate', 0.21)
    total_assets = balance.get('totalAssets')
    current_liabilities = balance.get('totalCurrentLiabilities', 0)

    if total_assets is None:
        raise ValueError("No se encontró totalAssets en balance sheet.")

    capital_empleado = total_assets - current_liabilities
    nopat = EBIT * (1 - tax_rate)
    roic = nopat / capital_empleado

    wacc = metrics.get('wacc', 0.09)

    return round(roic, 4), round(wacc, 4), round(nopat, 2), round(capital_empleado, 2)
roic, wacc, nopat, capital_empleado = calcular_roic_wacc_fmp(income_statement, balance_sheet, ratios, key_metrics)
print(f"ROIC: {roic*100:.2f}%, WACC: {wacc*100:.2f}%")

ROIC: 11.91%, WACC: 9.00%


In [4]:
def calcular_eva(nopat, wacc, capital_empleado):
    eva = nopat - (wacc * capital_empleado)
    return round(eva, 2)

eva = calcular_eva(nopat, wacc, capital_empleado)
print(f"EVA: {eva:,.2f}")


EVA: 2,565,880,600.70


In [5]:
def analisis_dupont(income, balance):
    net_income = income['netIncome']
    revenue = income['revenue']
    total_assets = balance['totalAssets']
    equity = balance['totalStockholdersEquity']

    roe = net_income / equity
    pm = net_income / revenue
    at = revenue / total_assets
    fl = total_assets / equity

    return round(roe, 4), round(pm, 4), round(at, 4), round(fl, 4)

roe, pm, at, fl = analisis_dupont(income_statement[0], balance_sheet[0])

print(
    f"ROE: {roe * 100:.2f}%\n"
    f"Margen Neto: {pm * 100:.2f}%\n"
    f"Rotación Activos: {at:.2f}\n"
    f"Apalancamiento: {fl:.2f}"
)

ROE: 17.62%
Margen Neto: 14.09%
Rotación Activos: 0.55
Apalancamiento: 2.28


In [6]:
print("RESUMEN DE RENTABILIDAD FINANCIERA")
print("="*40)
print(f"Ticker analizado: {ticker}")

# ROIC vs WACC
print(f"   ▪ ROIC: {roic*100:.2f}%")
print(f"   ▪ WACC estimado: {wacc*100:.2f}%")
print(f"   ▪ Valor Económico Agregado (EVA): ${eva:,.2f}")

# Análisis DuPont
print("\nAnálisis DuPont (descomposición del ROE):")
print(f"   ▪ ROE: {roe*100:.2f}%")
print(f"   ▪ Margen Neto (PM): {pm*100:.2f}%")
print(f"   ▪ Rotación de Activos (AT): {at:.2f}")
print(f"   ▪ Apalancamiento Financiero (FL): {fl:.2f}")


RESUMEN DE RENTABILIDAD FINANCIERA
Ticker analizado: BHP
   ▪ ROIC: 11.91%
   ▪ WACC estimado: 9.00%
   ▪ Valor Económico Agregado (EVA): $2,565,880,600.70

Análisis DuPont (descomposición del ROE):
   ▪ ROE: 17.62%
   ▪ Margen Neto (PM): 14.09%
   ▪ Rotación de Activos (AT): 0.55
   ▪ Apalancamiento Financiero (FL): 2.28
